<a href="https://colab.research.google.com/github/sumitsidana/corise-nlp/blob/main/Copy_of_NLP_Week_3_FAISS_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.


# Week 3: FAISS Tutorial

### What we are looking at
The goal of this small tutorial, is to provide you a quick overview into what FAISS does and how you can utilize it for Week 3 project. FAISS is an index for efficiently storing searchable embeddings of objects (e.g. sentences, images, ...). This efficient storing allows us to quickly compare our current object against the objects present in the index, and thus find relevant similar results. FAISS uses approximate nearest neighbor search to achieve these quick results.

### Instructions

1. Go through all the steps and look at what kind of embeddings we create.
1. Feel free to add more sentences to be embedded.
1. Make sure to have a look at the interactive graph, and see how close some results are, and how some are not. Does it make sense?
1. Have a look at the results retrieved from the FAISS index we made. Are they appropriate? Try and play around with the number of results it retrieves.

### Code Overview

- Dependencies: Install and import python dependencies
- Dataset creation
- Cohere API
- Creating a FAISS index


# Dependencies

✨ Now let's get started! To kick things off, as always, we will install some dependencies.

In [1]:
%%capture
!pip install cohere umap-learn faiss-gpu altair
!apt install libopenblas-base libomp-dev

Import all the necessary libraries we need throughout the project. Make sure to create a Cohere account and create an API key: https://os.cohere.ai/

In [5]:
import cohere
import umap
import faiss

import numpy as np
import pandas as pd
import altair as alt


COHERE_API_KEY = "PJWSIYnprK3KBftRo9W6lSXMYyA4FRcQCWrTChjv"
co = cohere.Client(COHERE_API_KEY)

## Dataset creation

Below we create our own small dataset, and its WONDERFUL🤩. Please feel free to add your own examples to it, the more the better✨✨! We make use of Cohere to quickly retrieve sentence embeddings that can be used for storing in our FAISS index.

In [7]:
sentences = [
             # Movies
             "I am watching a movie.",
             "I'm going to the movies.",
             "Cinema's popcorn smell is amazing.",
             "These guys kept talking while I was watching the movie.",
             # Groceries
             "Groceries are expensive now?",
             "What happend to all my groceries, they are all rotten.",
             "I like avocado toast",
             "Cheese is over there!",
             "Spinach is the food of the gods.",
             "Healthy dose of protein powder is always good.",
             # Music
             "Coldplay is not my favorite band anymore.",
             "I really liked MTV, with all the video clips.",
             "What music would you like me to play?",
             "He's playing piano very well."
             ]

df = pd.DataFrame (sentences, columns = ['conversation'])
df.head()

,conversation
0,I am watching a movie.
1,I'm going to the movies.
2,Cinema's popcorn smell is amazing.
3,These guys kept talking while I was watching t...
4,Groceries are expensive now?


## Cohere API: U there?
Here we retrieve the sentence embeddings through Cohere API. Be sure to check out the documentation: https://docs.cohere.ai/api-reference/

In [8]:
# Retrieve the embeddings of our sentences by calling
# the API of Cohere.
embeds = co.embed(texts = sentences,
                       model = "small", 
                       truncate = "LEFT").embeddings

embeds = np.array(embeds)
embeds.shape

(14, 1024)

Below we make use of UMAP and altair. UMAP we use to reduce the dimensions of our embeddings (Small size is 1024 😵). No other way to plot it then using dimensionality reduction. With Altair we make an interactive plot.


Please hover over some of these points and see if you can identify a pattern.

In [9]:
# UMAP reduces dimensions from 1024 to 2, which we can plot
reducer = umap.UMAP()
umap_embeds = reducer.fit_transform(embeds)
# Make interactive plot
df_explore = pd.DataFrame(data={'text': df['conversation']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=alt.X('x', scale=alt.Scale(zero=False)),
    y=alt.Y('y', scale=alt.Scale(zero=False)),
    tooltip=['text']
).properties(width=700, height=400)
chart.interactive()

/usr/local/lib/python3.8/dist-packages/umap/umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


alt.Chart(...)

## Creating FAISS: the good stuff.
Creating FAISS is rather straightforward. 
1. Identify which index you want to use, with the dimension your embeddings have. 
1. Add all the embeddings you want to add.

Since we made embeddings of sentences, we can now query this index with an example like *"I like eating cabbage"*. We turn this into a embedding and search for related sentences in our small index.

In [10]:
# Create our Approximate Nearest Neighbour Index (ANN)
# https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index
faiss_index = faiss.IndexFlatIP(1024)

# Convert from float64 to float32 to prevent bug:
# https://github.com/facebookresearch/faiss/issues/461
faiss_index.add(np.float32(np.stack(embeds)))

# Create an embedding for our sentence
embed = co.embed(texts = ["I like eating cabbage!"], 
                 model = "small",
                 truncate = "LEFT").embeddings

# Get the results
scores, indices = faiss_index.search(np.float32(np.array(embed)), 5)

# Print the results
for indice, score in zip(indices[0], scores[0]):
  print(sentences[indice], "\t\t\t\t", score)

I like avocado toast 				 1615.6023
Spinach is the food of the gods. 				 1347.9639
Cheese is over there! 				 970.35675
Cinema's popcorn smell is amazing. 				 938.289
What happend to all my groceries, they are all rotten. 				 923.1715


✨ Tada ✨, hopefully the results match your expectations! 

🙌 Good luck with the project! 🙌